# 13.4 TF 변환

- RAM이 충분히 작다면 전처리에 cache() 메서드 사용
- 데이터가 아주 큰 경우 : 
    - 아파치 빔, 스파크 같은 도구 사용하여 배포
        - 그러나 여러 도구를 사용하는 것은 유지/보수를 어렵게 함
        - 버그나 성능 감소

    - 전처리 담당하는 층을 동적으로 추가

    - TF 변환 :★ 전처리 한 번만 처리 가능
        - TFX(tensorflow extended의 일부분)
        - 텐서플로와 함께 제공되지 않음 --> TFX 따로 설치
        - TF 변환함수(스케일링, 버킷할당 등)를 사용하여 전처리 함수를 한 번만 정의해줌

In [3]:
# 두 개의 특성을 전처리하는 함수

import tensorflow_transform as tft

def preprocess(inputs):
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    standardized_age = tft.scale_to_z_score(median_age)
    ocean_proximity_id = tft.comput_and_apply_vocabulary(ocean_proximity)
    return{
        "standardized_median_age" : standardized_age,
        "ocean_proximity_id" : ocean_proximity_id
    }

# 아파치 빔을 사용해 preprocess() 함수를 전체 훈련 세트에 적용

- 위 예시에서 두개 특성의 어휘 사전을 계산함
- 해당 특성의 어휘 사전을 계산하는 컴포넌트를 "애널라이저"라 함
    - TF함수는 아파치 빔에서 계산에 필요한 통계(평균/표준편차/어휘 사전)에 해당하는 상수를 가짐


# 13.5 텐서플로 데이터셋(TFDS) 프로젝트

- 표준 데이터셋을 사용하고 싶은 경우
    - 텐서플로 데이터셋 [https://tensorflow.org/datasets] : 오디오, 비디오셋, MNIST 등 각종 데이터셋 제공  
        - 데이터셋에 대한 설명과 리스트 [https://homl.info/tfds]
    - tensorflow-datasets 라이브러리 설치 해야함 : TFDS는 텐서플로 기본에 포함되어 있지 않음
    - tfds.load()함수로 데이터를 다운로드 후 딕셔너리로 반환


In [12]:
# tfds.load()함수로 데이터를 다운로드 후 딕셔너리로 반환

import tensorflow_datasets as tfds
dataset = tfds.load(name="mnist")
mnist_train, mnist_test = dataset["train"], dataset["test"]

In [13]:
# 원하는 변환 : 셔플링, 배치 나누기, 매핑, 프리페칭 등 
mnist_train = mnist_train.shuffle(10000).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)

In [14]:
# load 시, as_supervised = True로 지정하면 레이블된 데이터셋 변환 (레이블된 데이터셋에서만 사용가능)
dataset = tfds.load(name="mnist", batch_size=32, as_supervised=True)
mnist_train = dataset["train"].prefetch(1)